In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

In [3]:
import sys
sys.argv = [""]

In [4]:
from cart_pole.utils import *
from cart_pole.agent import Agent
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import make_sync_vector_env, Storage
import pdb

In [8]:
args = parse_args(seconds_since_epoch=1764177877)
writer = init_tracking(args)
rng = init_rngs(args)

NameError: name 'SeedSequence' is not defined

In [38]:


    initialize()
storage = Storage(args, envs)

In [ ]:
global_step = 0
start_time = time.time()

In [39]:
rng = initialize_rngs(args)
seeds = [int(rng.uniform(0, 1E9)) for _ in range(args.num_envs)]

envs = make_sync_vector_env(args)
agent = Agent(envs).to(args.device)

optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

next_obs = torch.Tensor(envs.reset(seed=seeds)[0]).to(args.device)
next_terminated = torch.zeros(args.num_envs).to(args.device)
next_truncated = torch.zeros(args.num_envs).to(args.device)
num_updates = args.total_timesteps // args.batch_size
storage.reset(envs)

for _ in range(args.num_steps):

    storage.store("obs", next_obs)
    storage.store("terminated", next_terminated)
    storage.store("truncated", next_truncated)

    with torch.no_grad():
        action, logprob, _, value = agent.get_action_and_value(next_obs)
        storage.store("values", value.flatten())
        storage.store("actions", action)
        storage.store("logprobs", logprob)

    next_obs, reward, terminated, truncated, info =\
        envs.step(action.cpu().numpy())
    
    storage.store("rewards",
                  torch.tensor(reward).to(args.device).view(-1))

    next_obs = torch.Tensor(next_obs).to(args.device)
    next_terminated = torch.Tensor(terminated).to(args.device)
    next_truncated = torch.Tensor(truncated).to(args.device)

    storage.update_episode_info(info)

NameError: name 'initialize_rngs' is not defined

In [28]:
storage.episode_info

,cumulative_reward,episode_length
0,22,22
1,25,25
2,43,43
3,13,13


In [24]:
storage.terminated[:15,0].argwhere()

tensor([[12]], device='cuda:0')